# n-Pair Trident Scattering Process

In this file, we set up an n-pair trident scattering process. A trident
process looks like $k e^- \to e^- (e^- e^+)^n$.

You can download this file as a [jupyter notebook](trident.ipynb).

In [1]:
using QEDFeynmanDiagrams

We need some of the packages of the [QEDjl-project](https://github.com/QEDjl-project) for base
functionality and the `ScatteringProcess` type.

In [2]:
using QEDcore
using QEDprocesses

Let's decide how many pairs our trident should produce:

In [3]:
n = 2;

Now we setup the scattering process accordingly. We only consider a single spin/polarization
combination here. For an example with many spin and polarization combinations, refer to the
[n-photon Compton example](compton.md)

In [4]:
proc = ScatteringProcess(
    (Electron(), Photon()),                                                         # incoming particles
    (Electron(), ntuple(_ -> Electron(), n)..., ntuple(_ -> Positron(), n)...),     # outgoing particles
    (SpinUp(), PolX()),                                                             # incoming particle spin/pols
    (SpinUp(), ntuple(_ -> SpinUp(), 2 * n)...),                                    # outgoing particle spin/pols
)

generic QED process
    incoming: electron (spin up), photon (x-polarized)
    outgoing: electron (spin up), electron (spin up), electron (spin up), positron (spin up), positron (spin up)


The `feynman_diagrams` function returns an iterator for all possible Feynman diagrams
for this scattering process. With its `length` overload, we can check how many diagrams
there are.

In [5]:
length(feynman_diagrams(proc))

252

Next, we can generate the DAG representing the computation for our scattering process'
squared matrix element. This uses `ComputableDAGs.jl`.

In [6]:
dag = generate_DAG(proc)

Graph:
  Nodes: Total: 619, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 42, QEDFeynmanDiagrams.ComputeTask_Propagator: 42, 
         QEDFeynmanDiagrams.ComputeTask_Triple: 81, ComputableDAGs.DataTask: 334, QEDFeynmanDiagrams.ComputeTask_CollectTriples: 1, 
         QEDFeynmanDiagrams.ComputeTask_Pair: 69, QEDFeynmanDiagrams.ComputeTask_CollectPairs: 42, QEDFeynmanDiagrams.ComputeTask_BaseState: 7, 
         QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1
  Edges: 950
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


To continue, we will need [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl). Since `ComputableDAGs.jl` uses
`RuntimeGeneratedFunction`s as the return type of `ComputableDAGs.get_compute_function`, we need
to initialize it in our current module.

In [7]:
using ComputableDAGs
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

With the DAG, the process, and `RuntimeGeneratedFunctions` initalized,
we can now generate the actual computable function:

In [8]:
func = get_compute_function(dag, proc, cpu_st(), @__MODULE__);

Now we need an input for the function, which is a `QEDcore.PhaseSpacePoint`.
For now, we generate random momenta for every particle. In the future, QEDevents
will be able to generate physical `PhaseSpacePoint`s.

In [9]:
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...),
)

PhaseSpacePoint:
    process: generic QED process "ek -> eeepp"
    model: perturbative QED
    phasespace definition: spherical coordinates in electron rest frame
    incoming particles:
     -> incoming electron: [0.5568661453720338, 0.8878179529697953, 0.4919694180425218, 0.8525898568412901]
     -> incoming photon: [0.8045303656920232, 0.5301968737071986, 0.10076478173770831, 0.9009151055046453]
    outgoing particles:
     -> outgoing electron: [0.11189612668590732, 0.827624808561227, 0.978662359018709, 0.13596875466431646]
     -> outgoing electron: [0.29899255375228173, 0.14478095259548818, 0.27377066740494904, 0.20821183868521465]
     -> outgoing electron: [0.15975471941637887, 0.017338520089656417, 0.2159517040779959, 0.04662397434226728]
     -> outgoing positron: [0.7221731274089104, 0.5214706987875711, 0.3872645419876475, 0.33326186494610055]
     -> outgoing positron: [0.18513962098116854, 0.7942800745815964, 0.12348232047785579, 0.11209645180375905]


Finally, we can test that the function actually runs and computes something by
simply calling it on the `PhaseSpacePoint`:

In [10]:
func(psp)

0.9949412402845251

If we want, we can benchmark the execution speed too:

In [11]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.265 μs … 35.687 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.546 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.776 μs ±  1.329 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▁                                                          
  ▆██▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▂▂▂▂▁▂▁▂▂▂▂▂▁▂▂▂▁▂▂▂▂ ▂
  13.3 μs         Histogram: frequency by time        22.6 μs <

 Memory estimate: 2.56 KiB, allocs estimate: 24.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*